In [32]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [34]:
%cd drive/Colab Notebooks/drive

/content/drive/Colab Notebooks/drive/drive/Colab Notebooks/drive


In [0]:
import gensim
import nltk
import json
import numpy

In [0]:
def transfer(sentence,model):
    words=nltk.tokenize.word_tokenize(sentence)
    words=[word.lower() for word in words if word.isalpha()]
    vector=numpy.zeros(300)
   
    for word in words:
        try:
            temp=model[word]
            vector=vector+temp
        except:
            continue
    return vector.tolist()

In [37]:
model = gensim.models.KeyedVectors.load_word2vec_format('model.txt',binary=False)
nltk.download('punkt')    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
with open('test-unlabelled_evidence_0.8_siameseLSTM_epoch9_with_sentence.json','r') as f:
    file_content=json.load(f)

for key in file_content.keys():
    detail_content = file_content[key]
    claim = detail_content['claim']
    detail_content['claim'] = transfer(claim,model)

    evidences=detail_content['evidence']
    for evidence in evidences:
      if(evidence!=[]):
        evidence[2]=transfer(evidence[2],model)

In [0]:
def account(evidences):
    vector=numpy.zeros(300)
    for evidence in evidences:
        vector=vector+evidence[2]
    return vector

In [0]:
test_x=[]
for key in file_content.keys():
    detail_content = file_content[key]
    claim = numpy.array(detail_content['claim'])
    evidence=account(detail_content['evidence'])
    temp=claim-evidence

    test_x.append(temp)
 

In [0]:
from sklearn.externals import joblib
clf=joblib.load('svc_model.pkl')
test_y=clf.predict(test_x)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator SVC from version 0.20.3 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
with open('test-unlabelled_evidence_0.8_siameseLSTM_epoch9_with_sentence.json','r') as f:
    file_content=json.load(f)

for i,key in enumerate(file_content.keys()):
    detail_content = file_content[key]
    if(test_y[i]==1):
        detail_content['label']='SUPPORTS'
    elif(test_y[i]==0):
        detail_content['label']='REFUTES'
    else:
        detail_content['label']='NOT ENOUGH INFO'
     
    evidences=detail_content['evidence']
    for evidence in evidences:
      del evidence[2]
    
 

In [0]:
with open('word2vec-result.json','w') as t:
    json.dump(file_content, t, indent=2, separators=(',',':'))

In [0]:

"".join(["ad","sadsad"])